# Предсказания погоды

Используя датасет seattle-weather.csv предскажите погодные условия различными методами. Сравните разные типы классификаторов.

1) Входные данные:

* precipitation - величина осадков

* tempmax - максимальная дневная температура

* tempmin - минимальная дневная температура

* wind - скорость ветра

2) Погодные условия:

* drizzle - моросящий дождь, изморось

* rain - дождь

* sun - солнечно

* snow - снег

* fog - туман

Для теста использовать последние 400 строк датасета

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Подготовка данных

In [2]:
df = pd.read_csv("seattle-weather.csv")

In [3]:
# посмотрим как выглядят данные
df.head(10)

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
5,2012-01-06,2.5,4.4,2.2,2.2,rain
6,2012-01-07,0.0,7.2,2.8,2.3,rain
7,2012-01-08,0.0,10.0,2.8,2.0,sun
8,2012-01-09,4.3,9.4,5.0,3.4,rain
9,2012-01-10,1.0,6.1,0.6,3.4,rain


In [4]:
# посмотрим информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1461 non-null   object 
 1   precipitation  1461 non-null   float64
 2   temp_max       1461 non-null   float64
 3   temp_min       1461 non-null   float64
 4   wind           1461 non-null   float64
 5   weather        1461 non-null   object 
dtypes: float64(4), object(2)
memory usage: 68.6+ KB


In [5]:
# посмотрим распределение классов погоды
df['weather'].value_counts()

,count
weather,
rain,641
sun,640
fog,101
drizzle,53
snow,26


In [6]:
# посмотрим есть ли пропуски
df.isnull().sum()

,0
date,0
precipitation,0
temp_max,0
temp_min,0
wind,0
weather,0


Предсказываем weather, удаляем лишние столбцы

In [7]:
y = df['weather']
X = df.drop(['date', 'weather'], axis=1)

In [8]:
X.head()

,precipitation,temp_max,temp_min,wind
0,0.0,12.8,5.0,4.7
1,10.9,10.6,2.8,4.5
2,0.8,11.7,7.2,2.3
3,20.3,12.2,5.6,4.7
4,1.3,8.9,2.8,6.1


In [9]:
y.head()

,weather
0,drizzle
1,rain
2,rain
3,rain
4,rain


Делим выборку так, чтобы 400 значений оказались в тесте

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = (X[0:1060], X[1061:], y[0:1060], y[1061:])

In [11]:
print("data:   ", df.shape[0])
print("X_train:", X_train.shape[0])
print("X_test: ", X_test.shape[0])

data:    1461
X_train: 1060
X_test:  400


In [12]:
from sklearn.metrics import accuracy_score, classification_report

### 1. DummyClassifier

Делает прогнозы, которые игнорируют входные объекты (отвечает рандомно, по нескольким простым стратегиям). Одна из стратегий - ответить самым часто встречающимся классом.

In [13]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier()
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)

In [17]:
pd.Series(y_pred).value_counts()

,count
rain,400


Классификатор просто отвечал самым частым вариантом - rain

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       0.40      1.00      0.57       161
        snow       0.00      0.00      0.00         1
         sun       0.00      0.00      0.00       173

    accuracy                           0.40       400
   macro avg       0.08      0.20      0.11       400
weighted avg       0.16      0.40      0.23       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Метрики очевидно не очень

## 2. KNeighborsClassifier

1) Вычислить расстояние от каждого объекта обучающей выборки до тестового объекта.

2) Найти k объектов обучающей выборки (соседей) с наименьшим расстоянием до тестового объекта.

3) Вернуть наиболее встречающийся класс среди k соседей.

Для каждого из значений в тестовой выборке находятся k ближайших соседей среди обучающей выборке в 4х мерном пространстве характеристик и возвращается наиболее их часто встречаемый класс.

In [20]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [21]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.20      0.03      0.06        58
        rain       0.83      0.88      0.85       161
        snow       0.00      0.00      0.00         1
         sun       0.68      0.76      0.72       173

    accuracy                           0.69       400
   macro avg       0.34      0.33      0.33       400
weighted avg       0.66      0.69      0.66       400



если взять больше соседей:

In [30]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.67      0.07      0.12        58
        rain       0.81      0.88      0.84       161
        snow       0.25      1.00      0.40         1
         sun       0.70      0.84      0.77       173

    accuracy                           0.73       400
   macro avg       0.49      0.56      0.43       400
weighted avg       0.73      0.73      0.69       400



In [26]:
clf = KNeighborsClassifier(n_neighbors=9)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [27]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       0.80      0.88      0.84       161
        snow       1.00      1.00      1.00         1
         sun       0.68      0.86      0.76       173

    accuracy                           0.73       400
   macro avg       0.49      0.55      0.52       400
weighted avg       0.62      0.73      0.67       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Метрики лучше чем у Dummy, нужно аккуратно подбирать число соседей и получить неплохой результат

## 3. Наивная байесовская классификация (Гауссовский вариант)

Наивные байесовские классификаторы относятся к семейству простых вероятностных классификаторов и берут начало из теоремы Байеса, которая применительно к данному случаю рассматривает функции как независимые (из-за чего названы наивными).

Преобразуем набор данных в частотную таблицу (frequency table).
Создадим таблицу правдоподобия (likelihood table), рассчитав соответствующие вероятности. Например, вероятность облачной погоды (overcast) составляет 0,29.
С помощью теоремы Байеса рассчитаем апостериорную вероятность для каждого класса при данных погодных условиях. Класс с наибольшей апостериорной вероятностью будет результатом прогноза.

In [32]:
from sklearn.naive_bayes import GaussianNB

gauss = GaussianNB()
gauss.fit(X_train, y_train)
y_pred = gauss.predict(X_test)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       1.00      0.98      0.99       161
        snow       0.20      1.00      0.33         1
         sun       0.73      0.98      0.83       173

    accuracy                           0.82       400
   macro avg       0.39      0.59      0.43       400
weighted avg       0.72      0.82      0.76       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Метрики еще лучше, но редкие классы мы по-прежнему находим плохо

## 4. Логистическая регрессия

Логистическая регрессия — это метод статистического анализа, используемый для бинарной классификации. Она оценивает вероятность принадлежности объекта к определенному классу, основываясь на одной или нескольких независимых переменных. Основная идея заключается в использовании логистической функции (сигмоиды), которая преобразует линейную комбинацию входных признаков в значение от 0 до 1. Если вероятность превышает порог (обычно 0.5), объект классифицируется как принадлежащий к положительному классу.

In [82]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(multi_class='ovr', n_jobs=-1)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [83]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       0.93      0.96      0.95       161
        snow       1.00      1.00      1.00         1
         sun       0.71      0.95      0.81       173

    accuracy                           0.80       400
   macro avg       0.53      0.58      0.55       400
weighted avg       0.68      0.80      0.74       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Метрики чуть похуже, но стоит учесть, что логистическая регрессия очень простая и быстрая

## 5. SVM (метод опорных векторов)

Метод опорных векторов (SVM) — это мощный алгоритм для классификации и регрессии. Он работает, находя гиперплоскость, которая максимально разделяет классы в пространстве признаков. SVM может использовать различные ядра (линейные, полиномиальные, радиальные) для работы с нелинейными данными, преобразуя их в более высокое пространство признаков, где они могут быть линейно разделимы. Основная цель SVM — максимизировать расстояние между ближайшими точками разных классов (опорными векторами) и гиперплоскостью.

In [80]:
from sklearn.svm import SVC

svm_model = SVC()
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

In [81]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       0.95      0.87      0.91       161
        snow       1.00      1.00      1.00         1
         sun       0.66      0.97      0.79       173

    accuracy                           0.77       400
   macro avg       0.52      0.57      0.54       400
weighted avg       0.67      0.77      0.71       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Метрики похуже, это значит, что разделить наши классы линейно проблематично

## 6. Стохастический градиентный спуск

Градиентный спуск — метод нахождения минимального значения функции потерь (существует множество видов этой функции). Минимизация любой функции означает поиск самой глубокой впадины в этой функции. Функция используется, чтобы контролировать ошибку в прогнозах модели машинного обучения. Поиск минимума означает получение наименьшей возможной ошибки или повышение точности модели. Мы увеличиваем точность, перебирая набор учебных данных при настройке параметров нашей модели (весов и смещений).

*Градиентный спуск нужен для минимизации функции потерь.*

Суть алгоритма – процесс получения наименьшего значения ошибки. Аналогично это можно рассматривать как спуск во впадину в попытке найти золото на дне ущелья (самое низкое значение ошибки).

Градиент функции $f(x) = f(x_1, \dots, x_d)$ от многих переменных в точке $x_0$ - это вектор ее частных производных, вычисленных в точке $x_0$.
$$\nabla_x f \bigl | _{x_0} = \biggl(\frac{\partial f}{\partial x_1}, \dots, \frac{\partial f}{\partial x_d} \biggr ) \biggl | _{x_0}$$

У большинства моделей возникает проблема с коэффициентом $alpha$ скорости обучения, который нужно осторожно подобрать. Если этот размер шага $alpha$ слишком велик, мы преодолеем минимум: то есть промахнемся мимо минимума. Если $alpha$ слишком мала, мы потратим слишком много итераций, чтобы добраться до минимума.

![image](https://drive.google.com/uc?export=view&id=1FUhNB533Cs6xlAFa2mlCM9ilts5QdHin)


Проблема обычного градиентного спуска заключается в том, что для того чтобы определить новые веса модели, необходимо вычислить градиент от каждого элемента выборки, что может сильно замедлять алгоритм.

Стохастический градиентный спуск дает преймущество в скорости - значение градиента вычисляется только на одном случайном элементе обучения. Таким образом параметры модели изменяются после каждого объекта обучения. Для больших массивов данных стохастический градиентный спуск может дать значительное преимущество в скорости по сравнению со стандартным градиентным спуском, который вычисляет градиент из полного набора данных.

С каждой итерацией спуска, нужно перемешать набор обучения и выбрать случайный пример обучения. Поскольку вы используете только один случайный пример для обучения, ваш путь к локальным минимумам будет очень шумным, как у нетрезвого человека.

In [34]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(n_jobs=-1)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       0.93      0.95      0.94       161
        snow       1.00      1.00      1.00         1
         sun       0.72      0.94      0.81       173

    accuracy                           0.79       400
   macro avg       0.53      0.58      0.55       400
weighted avg       0.69      0.79      0.73       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Метрики не на высоте. Попробуем подобрать парметры для улучшения характеристик

In [47]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Определяем параметры для перебора
param_grid = {
    'loss': ['hinge', 'log_loss', 'squared_hinge', 'perceptron'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [1e-4, 1e-3, 1e-2],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.1, 0.2, 0.3],
}

# Создаем экземпляр SGDClassifier
sgd = SGDClassifier(n_jobs=-1)

# Создаем GridSearchCV
grid_search = GridSearchCV(estimator=sgd, param_grid=param_grid,
                           scoring='accuracy', cv=5, n_jobs=-1)

# Обучаем модель с перебором параметров
grid_search.fit(X_train, y_train)

# Получаем лучшие параметры и модель
best_model = grid_search.best_estimator_

# Делаем предсказания на тестовой выборке
y_pred = best_model.predict(X_test)

In [48]:
# Выводим classification report для лучшей модели
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.67      0.03      0.07        58
        rain       0.82      1.00      0.90       161
        snow       0.00      0.00      0.00         1
         sun       0.78      0.90      0.84       173

    accuracy                           0.80       400
   macro avg       0.45      0.39      0.36       400
weighted avg       0.76      0.80      0.73       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Метрики немного улучшились, иногда видно даже редкие классы

## 7. Деревья решений - DecisionTreeClassifier и RandomForestClassifier



Решающее дерево предсказывает значение целевой переменной с помощью применения последовательности простых решающих правил (которые называются предикатами). Этот процесс в некотором смысле согласуется с естественным для человека процессом принятия решений.

Хотя обобщающая способность решающих деревьев невысока, их предсказания вычисляются довольно просто, из-за чего решающие деревья часто используют как кирпичики для построения ансамблей — моделей, делающих предсказания на основе агрегации предсказаний других моделей.

**RandomForest и Bagging - ансамбли таких деревьев**

`RandomForest` - ответы усредняются, в задаче классификации принимается решение голосованием по большинству

`Bagging` – собирает усложнённые классификаторы, при этом параллельно обучая базовые (улучшающее объединение)

### Попробуем одно дерево

In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [50]:
des_tree = DecisionTreeClassifier()
des_tree.fit(X_train, y_train)
y_pred = des_tree.predict(X_test)

In [51]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.22      0.07      0.11        58
        rain       0.91      0.96      0.93       161
        snow       0.12      1.00      0.22         1
         sun       0.74      0.82      0.78       173

    accuracy                           0.75       400
   macro avg       0.40      0.57      0.41       400
weighted avg       0.72      0.75      0.73       400



### Теперь случайный лес

RF (random forest) — это множество решающих деревьев. В задаче классификации принимается решение голосованием по большинству. Все деревья строятся независимо по следующей схеме:

1) Генерируем при помощи бэггинга  B  выборок

2) Обучаем каждое решающее дерево на своей выборке

3) Получаем результат при помощи голосования большинства

Плюсы:

Можно осуществить параллельную реализацию
Не переобучается с ростом количества выборок  B
Минусы:

Менее интерпретируемый, чем решающее дерево
Деревья не исправляют ошибки друг друга

In [52]:
rf_clf = RandomForestClassifier(n_jobs=-1)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)

In [53]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.40      0.03      0.06        58
        rain       0.97      0.99      0.98       161
        snow       0.50      1.00      0.67         1
         sun       0.74      0.97      0.84       173

    accuracy                           0.82       400
   macro avg       0.52      0.60      0.51       400
weighted avg       0.77      0.82      0.77       400



### BaggingClassifier

Идея бэггинга (bagging) заключается в следующем. Выберем из обучающей выборки $n$
 примеров. Будем выбирать примеры равновероятно, с повторением. Получим новую выборку $X^1$
, в которой некоторых элементов исходной выборки не будет, а какие-то могут войти несколько раз. С помощью некоторого алгоритма $b$
 обучим на этой выборке модель $b_1(x) = b(x, X^1)$
. Повторим процедуру: сформируем вторую выборку $X^2$
 из $n$
 элементов с возвращением и с помощью того же алгоритма обучим на ней модель $b_2(x) = b(x, X^2)$
. Повторив процедуру $k$
 раз, получим $k$
 моделей, обученных на $k$
 выборках. Чтобы получить одно предсказание, усредним предсказания всех моделей:

$$
\mathrm a(x) = \frac{1}{k}(b_1(x) + \dots + b_k(x)).
$$

Плюсы:
- Уменьшает переобучение, если базовые модели были переобучены (например, решающие деревья)

Минусы:
- Время обучения увеличивается в $k$ раз (по количеству обученных моделей)

In [54]:
bg_clf = BaggingClassifier(n_jobs=-1)
bg_clf.fit(X_train, y_train)
y_pred = bg_clf.predict(X_test)

In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.47      0.14      0.21        58
        rain       0.96      0.98      0.97       161
        snow       0.25      1.00      0.40         1
         sun       0.76      0.92      0.83       173

    accuracy                           0.81       400
   macro avg       0.49      0.61      0.48       400
weighted avg       0.78      0.81      0.78       400



### Попробуем немного ограничить количество значений в листе в случайном лесу

In [56]:
rf_clf = RandomForestClassifier(
    criterion="entropy", min_samples_leaf=3, n_jobs=-1
  )
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)

In [57]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       1.00      0.02      0.03        58
        rain       1.00      1.00      1.00       161
        snow       1.00      1.00      1.00         1
         sun       0.73      1.00      0.84       173

    accuracy                           0.84       400
   macro avg       0.75      0.60      0.58       400
weighted avg       0.87      0.84      0.77       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Итого

Случайный лес, несмотря на простоту, показывает хорошие метрики

## 8. Градиентный бустинг

**Бустинг (boosting)** – это ансамблевый метод, в котором так же, как и в методах выше, строится множество базовых алгоритмов из одного семейства, объединяющихся затем в более сильную модель. Отличие состоит в том, что в бэггинге и случайном лесе базовые алгоритмы учатся независимо и параллельно, а в бустинге – последовательно.

Каждый следующий базовый алгоритм в бустинге обучается так, чтобы уменьшить общую ошибку всех своих предшественников. Как следствие, итоговая композиция будет иметь меньшее смещение, чем каждый отдельный базовый алгоритм. Поскольку основная цель бустинга – уменьшение смещения, в качестве базовых алгоритмов часто выбирают алгоритмы с высоким смещением и небольшим разбросом. Например, если в качестве базовых классификаторов выступают деревья, то их глубина должна быть небольшой – обычно не больше 2-3 уровней.

Ещё одной важной причиной для выбора моделей с высоким смещением в качестве базовых является то, что такие модели, как правило, быстрее учатся. Это важно для их последовательного обучения, которое может стать очень дорогим по времени, если на каждой итерации будет учиться сложная модель.

На текущий момент основным видом бустинга с точки зрения применения на практике является градиентный бустинг. Хотя случайный лес – мощный и достаточно простой для понимания и реализации алгоритм, на практике он чаще всего уступает градиентному бустингу. Поэтому градиентный бустинг сейчас – основное продакшн-решение, если работа происходит с табличными данными (в работе с однородными данными – картинками, текстами – доминируют нейросети).

In [74]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(
    max_depth=3, learning_rate=0.1, n_estimators=10
  )
gb_clf.fit(X_train, y_train)
y_pred = gb_clf.predict(X_test)

In [75]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       1.00      0.99      0.99       161
        snow       0.33      1.00      0.50         1
         sun       0.73      1.00      0.84       173

    accuracy                           0.83       400
   macro avg       0.41      0.60      0.47       400
weighted avg       0.72      0.83      0.77       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Видим очень хорошие метрики, но маленькие классы по-прежнему не видны

#### Итак, метод ансамблей – более мощный инструмент по сравнению с отдельно стоящими моделями прогнозирования, поскольку:

- он сводит к минимуму влияние случайностей, усредняя ошибки каждого базового классификатора;
- уменьшает дисперсию, поскольку несколько разных моделей, исходящих из разных гипотез, имеют больше шансов прийти к правильному результату, чем одна отдельно взятая.


## 9. CatBoost

**Cat Boost** - алгоритм градиентного бустинга решающих деревьев.

Разработан в компании Яндекс и используется в поиске, рекомендательных системах, голосовых помощниках, беспилотных автомобилях, предсказаниях погоды, и т.д. не только внутри компании, но и в CERN, Cloudflare, и др.

Практически любой современный метод на основе градиентного бустинга работает с числовыми признаками.

Если в наборе данных присутствуют не только числовые, но и категориальные признаки, то необходимо переводить категориальные признаки в числовые. Это приводит к искажению их сути и потенциальному снижению точности работы модели. Именно поэтому было важно разработать алгоритм, который умеет работать не только с числовыми признаками, но и с категориальными напрямую, закономерности между которыми этот алгоритм будет выявлять самостоятельно, без ручной «помощи». CatBoost — библиотека для градиентного бустинга, главным преимуществом которой является то, что она одинаково хорошо работает «из коробки» как с числовыми признаками, так и с категориальными.

Использует небрежные (oblivious) деревья решений, чтобы вырастить сбалансированное дерево. Одни и те же функции используются для создания левых и правых разделений (split) на каждом уровне дерева.

In [70]:
!pip install catboost -q
from google.colab import output
output.enable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.7 MB/s eta 0:00:00


In [76]:
from catboost import CatBoostClassifier

cbst = CatBoostClassifier(verbose=0) # лучше указать параметры, но воспользуемся автоподбором
cbst.fit(X_train, y_train)
y_pred = cbst.predict(X_test)

In [77]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.50      0.05      0.09        58
        rain       0.98      0.99      0.98       161
        snow       0.50      1.00      0.67         1
         sun       0.74      0.96      0.83       173

    accuracy                           0.82       400
   macro avg       0.54      0.60      0.52       400
weighted avg       0.79      0.82      0.77       400



Метрики чуть похуже предыдущего классификатора, но зато стали появляться редкие классы

## 10. Многослойный персептрон MLP Classifier

Персептрон — это один из первых алгоритмов для обучения нейронных сетей, предназначенный для бинарной классификации. Он состоит из одного слоя нейронов и использует линейную функцию активации. Персептрон обучается с помощью алгоритма, который корректирует веса на основе ошибок предсказания. Хотя персептрон может решать только линейно разделимые задачи, он стал основой для более сложных многослойных нейронных сетей, которые способны решать более сложные задачи.

In [90]:
from sklearn.neural_network import MLPClassifier
# Создание MLP с двумя скрытыми слоями
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), alpha=1, max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

In [91]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     drizzle       0.00      0.00      0.00         7
         fog       0.00      0.00      0.00        58
        rain       1.00      0.99      0.99       161
        snow       1.00      1.00      1.00         1
         sun       0.72      1.00      0.84       173

    accuracy                           0.83       400
   macro avg       0.54      0.60      0.57       400
weighted avg       0.72      0.83      0.76       400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Мы видим едва ли не лучшие метрики, сравнимые с градиентным бустингом и CatBoost. Данное наблюдение открывает нам дорогу в мир глубинного обучения и нейронных сетей

## Вывод

Лучше всего себя показывают ансамблевые модели - случайный лес и грандиентный бустинг, так как они сводят к минимуму влияние случайностей, корректируя или усредняя ошибки каждого базового классификатора, и уменьшают дисперсию, поскольку несколько разных моделей, исходящих из разных гипотез, имеют больше шансов прийти к правильному результату, чем одна отдельно взятая. Такие же хорошие метрики показывает многослойный персептрон, так как он корректирует веса на основе ошибок предсказания, открывая дорогу в глубинное обучение.